### The goal of this notebook is to find if we can extract the following information from our dataset.

1. For each Associated action do we have the action labels.
2. If we have Associated action labels for each frame do we have information about the start and end frames of each action.

In [4]:
frames_path = "/Users/jyothivishnuvardhankolla/Desktop/CS-7180 Advanced Perception/Physical_Retail_Challenge/Data/activity_based/anns"
anns_path = "/Users/jyothivishnuvardhankolla/Desktop/CS-7180 Advanced Perception/Physical_Retail_Challenge/Data/activity_based/anns"

**It is given in readme that each frame has an unique id so lets check the total number of frames and total number of unique id's present**

**Why do this?**
if the number of unique id's are less than the number of frames it means we can extract information about start and end frames for a particular action in the untrimmed video

In [5]:
import os

def count_unique_ids(anns_paths):
    unique_ids = set()

    for file in os.listdir(anns_paths):
        if file.endswith('.txt'):
            file_path = os.path.join(anns_path, file)
            with open(file_path, 'r') as file:
                for line in file:
                    data = line.strip().split(',')
                    unique_id = data[0].strip('[]')
                    unique_ids.add(unique_id)

    return len(unique_ids)

print(f"Total number of unique id's {count_unique_ids(anns_path)} and total number of files is {len(os.listdir(anns_path))}")

Total number of unique id's 4726 and total number of files is 73683


**Now as we have found The number of unique_id's is very less than the total number of files now let's prepare a input txt file for the TAL problem.**

**Data Format:**
1. For each unique action in the video sequence temporally assign a data point as (start_frame, end_frame, unique_id, class)
2. Visualize each trimmed video to see if our logic was correct.

In [35]:
import os

def save_results(final_annotations_list, output_file):
    with open(output_file, 'w') as file:
        for data in final_annotations_list:
            line = f"{data[0]}, {data[1]}, {data[2]}, {data[3]}\n"
            file.write(line)


def create_TAL_annotations(anns_path: str):
    final_annotations = []
    skipped_frames = []
    last_unique_id, last_class, start_file, prev_file = None, None, None, None

    # Sort the files to ensure they are sorted in order.
    files = sorted([f for f in os.listdir(anns_path) if f.endswith('.txt')])

    for file in files:
        file_path = os.path.join(anns_path, file)
        with open(file_path, 'r') as f:
            line = f.readline()
            if not line:
                skipped_frames.append(file)
                continue
            data = line.strip().split(',')
            unique_id = data[0].strip('[]')
            action_class = int(data[1].strip())

        if last_unique_id is None:
            last_unique_id, last_class, start_file = unique_id, action_class, file
        elif unique_id != last_unique_id or action_class != last_class:
            # change detected record the clip
            final_annotations.append((start_file, prev_file, last_unique_id, last_class))
            last_unique_id, last_class, start_file = unique_id, action_class, file

        prev_file = file

    # Adding the last segment if it exists
    if last_unique_id is not None:
        final_annotations.append((start_file, prev_file, last_unique_id, last_class))

    return final_annotations
          


final_anns_list = create_TAL_annotations(anns_path=anns_path)

# save the final_annotations in a text file.
output_file = "/Users/jyothivishnuvardhankolla/Desktop/CS-7180 Advanced Perception/Physical_Retail_Challenge/Data/activity_based/final_anns.txt"
save_results(final_anns_list, output_file)
